# Iceberg with SQL Catalog

Example of Iceberg with SQL Glue Catalog with AWS S3 as Object Storage.

Database needs to exists on AWS Glue and user needs to have `AWSGlueConsoleFullAccess` permission.

More info [https://iceberg.apache.org/docs/latest/aws/#glue-catalog]

In [1]:
import os

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.master("spark://spark-master:7077")  # type: ignore
    .appName("Testing Iceberg with SQL Catalog")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .getOrCreate()
)

print(f'The PySpark {spark.version} version is running...')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/22 13:52:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The PySpark 3.5.0 version is running...


### Test Spark is running

In [2]:
data = [("John", 28), ("Alice", 35), ("Bob", 42)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.createOrReplaceTempView("tmp_people")
df.show()

+-----+---+
| Name|Age|
+-----+---+
| John| 28|
|Alice| 35|
|  Bob| 42|
+-----+---+



## Config Iceberg catalog

```bash
spark-sql \
    --packages org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.4.2,org.apache.iceberg:iceberg-aws-bundle:1.4.2 \
    --conf spark.sql.catalog.glue=org.apache.iceberg.spark.SparkCatalog \
    --conf spark.sql.catalog.glue.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog \
    --conf spark.sql.catalog.glue.warehouse=s3a://cloud-sdk/warehouse \
    --conf spark.sql.catalog.glue.io-impl=org.apache.iceberg.aws.s3.S3FileIO \
    --conf spark.hadoop.fs.s3a.access.key=${AWS_ACCESS_KEY_ID} \
    --conf spark.hadoop.fs.s3a.secret.key=${AWS_SECRET_ACCESS_KEY}
```

In [10]:
CATALOG_NAME = "demo"
DEFAULT_NAMESPACE = "default"

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID") or ""
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY") or ""

spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.warehouse", "s3a://cloud-sdk/warehouse/")
spark.conf.set(f"spark.sql.catalog.{CATALOG_NAME}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
spark.conf.set(f"spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID)
spark.conf.set(f"spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)

### Change Catalog

Activate the catalog and optionally the namespace.


In [24]:
%%sql
USE demo;

++
||
++
++

### Create new Namespace

A Namespace in Glue is a Database.

In [25]:
%%sql
CREATE NAMESPACE new_namespace;

++
||
++
++

### Activate namespace

In [26]:
%%sql
USE demo.new_namespace;

++
||
++
++

### List existing catalogs, namespaces, tables

In [27]:
%%sql
SHOW catalogs;

catalog
demo
spark_catalog


In [28]:
%%sql
SHOW namespaces;

namespace
demo
iceberg
new_namespace
test
`unique-stocks`


### Create table

In [29]:
%%sql
CREATE TABLE people (
  name STRING,
  age INT
) USING iceberg;

++
||
++
++

Create a table in a different Namespace than the one activated.

Glue needs to have the `catalog_name`.`database_name`.`table_name`.

In [33]:
%%sql
CREATE TABLE IF NOT EXISTS demo.demo.quote (date Date, Close Float) USING iceberg PARTITIONED BY (YEAR(date));

++
||
++
++

### Show tables

In [34]:
%%sql
SHOW tables;

namespace,tableName,isTemporary
new_namespace,people,False


## Insert

In [35]:
quote = spark.read.csv("data/GDAXI.INDX.csv", header=True, inferSchema=True).select(["Date", "Close"])
quote.writeTo("demo.demo.quote").append()

In [36]:
%%sql
-- Insert from temp view
INSERT INTO demo.demo.people SELECT * FROM tmp_people

++
||
++
++

## Query Data

In [40]:
%%sql
SELECT * FROM demo.demo.people;

name,age
John,28
Alice,35
Bob,42


In [39]:
%%sql
SELECT * FROM demo.demo.quote;

date,Close
2010-01-04,6048.2998046875
2010-01-05,6031.85986328125
2010-01-06,6034.330078125
2010-01-07,6019.35986328125
2010-01-08,6037.60986328125
2010-01-11,6040.5
2010-01-12,5943.0
2010-01-13,5963.14013671875
2010-01-14,5988.8798828125
2010-01-15,5875.97021484375


### Stop Spark

In [41]:
spark.stop()

# PyIceberg with Glue

In [48]:
import os

from pyiceberg.catalog import load_catalog


AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID") or ""
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY") or ""

catalog = load_catalog(
    "glue",
    **{
        "type": "glue",
        "region_name": "eu-central-1",
        "s3.access-key-id": AWS_ACCESS_KEY_ID,
        "s3.secret-access-key": AWS_SECRET_ACCESS_KEY,
    },
)

In [49]:
catalog.list_tables('demo')

[('demo', 'people'), ('demo', 'quote')]

In [50]:
catalog.load_table("demo.quote").schema()

ModuleNotFoundError: No module named 's3fs'